<a href="https://colab.research.google.com/github/leticiarccorrea/case-ticketsclassification/blob/main/case_ticketclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Objetivo de negócio: acelerar a distribuição (triagem) de tickets para o time correto, reduzindo tempo de fila e retrabalho.**

**Contexto dos chamados: dataset é composto de tickets de suporte de TI corporativo, com foco em infraestrutura, hardware, rede e software.**

A partir do objetivo do negócio e o contexto dos chamados, foram definidas 5 categorias de classificação de tickets:

1. Rede
Definição: Problemas relacionados à conectividade, infraestrutura de rede e acesso remoto.
Exemplo: Internet instável, VPN, Wi-Fi...

2. Computador (Hardware)
Definição: Falhas físicas ou de desempenho em equipamentos.
Exemplo: Notebook / desktop, Bateria, Tela...

3. Software & Sistema Operacional
Definição: Problemas em softwares instalados, IDEs, aplicações locais ou SO.
Exemplo: Erro em aplicações, IDEs (IntelliJ, VS Code, etc.), Atualizações de sistema...

4. Acesso, Permissões & Contas
Definição: Problemas de autenticação, autorização ou privilégios.
Exemplo: Login,Senha, Permissão negada, Acesso a sistemas...

5. Outros
Definição: Tudo que não se encaixa claramente nas categorias acima.


Além disso, cada uma dessas categorias terão uma subcateorica de criticidade: Urgente e Normal.

# **Pipeline da solução**

**1) Ingestão de dados (CSV)**
   
**2) Preparação e limpeza de texto**
   - concatena subject + body
   - normaliza (lowercase, remoção de ruído, etc.)
   - gera clean_text
   
**3) Pré-rotulagem (weak labels) via dicionário**
   - main_category {connectivity, computer_hardware, software_operating_system, access_permissions_accounts, other}
   - criticality {urgent, normal}
   ↓
**4) Split global (hold-out único)**
   - train/test estratificado por criticality
   - preserva índices para coerência entre modelos
   
**5) Feature engineering (vetorização TF-IDF)**
   - word n-grams (1–2)
   - char n-grams (3–5)
   - fit apenas no treino (evita data leakage)
   - concatenação (hstack) - matriz final de features
   ↓
**6) Treinamento de modelos**
   6.1) Criticidade (binário) – LinearSVC (dataset completo, inclui other)
   6.2) Categoria (multiclasse) – LinearSVC (treino sem other; other = fallback)
   ↓
**7) Avaliação e validação**
   - métricas no teste: accuracy, F1 macro, F1 urgent (criticidade)
   - matriz de confusão e classification report
   - diagnóstico de overfitting (train vs test)
   - validação cruzada (CV) no treino
   - interpretação: top features para o modelo de classes
   
**8) Predição em produção (função única)**
   predict_ticket(text):
     - aplica TF-IDF (transform)
     - prevê criticality
     - prevê main_category
     - aplica fallback: se margin < threshold → other
   
**9) Saída estruturada**
   { main_category, criticality, category_margin }


# 2. Import de bases

In [1]:
# import dataset

from google.colab import drive
import pandas as pd


# Access to Google Drive
drive.mount('/content/drive')
datapah = '/content/drive/MyDrive/casevoll/classificacao_atendimento.csv'

# Load file in pandas and spark
base = pd.read_csv(datapah)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
base.head()

,ticket_id,subject,body
0,2,Erro na Autocompletação de Código do IntelliJ ...,"Prezado Suporte ao Cliente <name>,\n\nEstou es..."
1,28,Problemas intermitentes de exibição após atual...,"Caro Suporte ao Cliente,\n\n Estou entrando em..."
2,45,Pedido de Assistência para Administração de Se...,"Prezado Suporte ao Cliente de Serviços de TI,\..."
3,46,NaN,"Prezado <name>,\n\nSentimos muito ao saber sob..."
4,63,Urgente: Relato de Falha na Bateria do MacBook...,"Prezado Atendimento ao Cliente,<br><br>Recente..."


In [3]:
base.count()

,0
ticket_id,473
subject,419
body,473


In [4]:
# import bibliotecas


# Standard library
import re
import html
import unicodedata
from dataclasses import dataclass
from typing import Dict, Any, Tuple

#
import numpy as np
import pandas as pd
from typing import Optional
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    accuracy_score
)
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack



# 3. Exploração e diagnóstico do dataset

In [5]:
#funções gerais

@dataclass
class ticket_dataset_quality_config:
    subject_column_name: str = "subject"
    body_column_name: str = "body"

    # short/low-info thresholds
    minimum_character_count: int = 60
    minimum_word_count: int = 10

    # generic messages / low-info patterns
    generic_message_patterns: Tuple[str, ...] = (
        r"\bpreciso de ajuda\b",
        r"\bajuda\b",
        r"\bn[aã]o funciona\b",
        r"\bnao funciona\b",
        r"\bcom problema\b",
        r"\berro\b",
        r"\bn[aã]o consigo\b",
        r"\bnao consigo\b",
        r"\burgente\b",  # used for criticidade signal
    )

    # near-duplicate computation cap
    near_duplicate_sample_size: int = 2500
    near_duplicate_similarity_threshold: float = 0.92
    tfidf_max_features: int = 50000
    tfidf_min_document_frequency: int = 2
    tfidf_max_document_frequency: float = 0.95

    # noise / thread / signature heuristics
    maximum_lines_to_consider: int = 200


# -----------------------------
# Common functions
# -----------------------------
def normalize_text(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = unicodedata.normalize("NFKC", text)
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def count_words(text: str) -> int:
    if not text:
        return 0
    return len(re.findall(r"\b\w+\b", text, flags=re.UNICODE))


def build_combined_text(base: pd.DataFrame, config: ticket_dataset_quality_config) -> pd.Series:
    subject_series = base[config.subject_column_name].fillna("").astype(str).str.strip()
    body_series = base[config.body_column_name].fillna("").astype(str).str.strip()
    combined = (subject_series + "\n\n" + body_series).map(normalize_text)
    return combined

In [6]:
# testando as funções acima

raw_text_test_cases = [
    None,
    "",
    "   ",
    "Erro no sistema",
    "Erro   grave    no   sistema",
    "Falha\r\nna\r\naplicação",
    "Sistema não abre\rTela branca",
    "Usuário relata erro ﬁ no login",
    "ＡＢＣ falha crítica",
    "Erro 404 detectado no módulo financeiro!!!",
    "   Solicitação de suporte urgente   ",
    "\n\n\nMuitas\n\n\nlinhas\n\n\n",
    "Problema intermitente\t\tno\t\tservidor",
    "ação rápida necessária",
]

processed_text_results = []

for raw_text in raw_text_test_cases:
    normalized_text = normalize_text(raw_text)
    word_count = count_words(normalized_text)

    # Critério mínimo: texto normalizado não vazio e com pelo menos 2 palavras
    if normalized_text and word_count >= 2:
        processed_text_results.append({
            "original_text": raw_text,
            "normalized_text": normalized_text,
            "word_count": word_count
        })

print(processed_text_results)

[{'original_text': 'Erro no sistema', 'normalized_text': 'Erro no sistema', 'word_count': 3}, {'original_text': 'Erro   grave    no   sistema', 'normalized_text': 'Erro grave no sistema', 'word_count': 4}, {'original_text': 'Falha\r\nna\r\naplicação', 'normalized_text': 'Falha\nna\naplicação', 'word_count': 3}, {'original_text': 'Sistema não abre\rTela branca', 'normalized_text': 'Sistema não abre\nTela branca', 'word_count': 5}, {'original_text': 'Usuário relata erro ﬁ no login', 'normalized_text': 'Usuário relata erro fi no login', 'word_count': 6}, {'original_text': 'ＡＢＣ falha crítica', 'normalized_text': 'ABC falha crítica', 'word_count': 3}, {'original_text': 'Erro 404 detectado no módulo financeiro!!!', 'normalized_text': 'Erro 404 detectado no módulo financeiro!!!', 'word_count': 6}, {'original_text': '   Solicitação de suporte urgente   ', 'normalized_text': 'Solicitação de suporte urgente', 'word_count': 4}, {'original_text': '\n\n\nMuitas\n\n\nlinhas\n\n\n', 'normalized_text'

In [7]:
# 3.1 Verificação de campos vazios (assunto e corpo)

def check_missing_ticket_fields(base: pd.DataFrame, config: ticket_dataset_quality_config) -> Dict[str, Any]:
    subject_series = base[config.subject_column_name]
    body_series = base[config.body_column_name]

    is_subject_empty = subject_series.isna() | (subject_series.astype(str).str.strip() == "")
    is_body_empty = body_series.isna() | (body_series.astype(str).str.strip() == "")
    is_both_empty = is_subject_empty & is_body_empty

    # "subject empty but body filled"
    is_subject_empty_body_filled = is_subject_empty & (~is_body_empty)

    # "generic subject + short body"
    subject_text = subject_series.fillna("").astype(str).str.lower().str.strip()
    body_text = body_series.fillna("").astype(str).map(normalize_text)
    body_word_count = body_text.map(count_words)

    generic_subject_patterns = (
        r"^ajuda$",
        r"^suporte$",
        r"^problema$",
        r"^erro$",
        r"^urgente$",
        r"^d[uú]vida$",
        r"^help$",
        r"^issue$",
    )
    is_generic_subject = subject_text.apply(
        lambda s: any(re.search(p, s, flags=re.IGNORECASE) for p in generic_subject_patterns)
    )
    is_body_very_short = body_word_count < config.minimum_word_count
    is_generic_subject_and_short_body = is_generic_subject & is_body_very_short & (~is_body_empty)

    summary = {
        "subject_empty_row_count": int(is_subject_empty.sum()),
        "body_empty_row_count": int(is_body_empty.sum()),
        "both_empty_row_count": int(is_both_empty.sum()),
        "subject_empty_body_filled_row_count": int(is_subject_empty_body_filled.sum()),
        "generic_subject_and_short_body_row_count": int(is_generic_subject_and_short_body.sum()),
        "subject_empty_ratio": float(is_subject_empty.mean()),
        "body_empty_ratio": float(is_body_empty.mean()),
        "both_empty_ratio": float(is_both_empty.mean()),
    }

    masks = {
        "is_subject_empty": is_subject_empty,
        "is_body_empty": is_body_empty,
        "is_both_empty": is_both_empty,
        "is_subject_empty_body_filled": is_subject_empty_body_filled,
        "is_generic_subject_and_short_body": is_generic_subject_and_short_body,
    }

    return {"summary": summary, "masks": masks}

In [8]:
# 3.2 Conteúdo muito curto ou pouco informativo

def check_low_information_tickets(
    base: pd.DataFrame, combined_text: pd.Series, config: ticket_dataset_quality_config
) -> Dict[str, Any]:
    character_count = combined_text.map(len)
    word_count = combined_text.map(count_words)

    is_very_short = (character_count < config.minimum_character_count) | (word_count < config.minimum_word_count)

    # Generic content indicator: matches generic phrases OR extremely low vocabulary diversity
    def generic_hit(text: str) -> bool:
        t = (text or "").lower()
        return any(re.search(p, t, flags=re.IGNORECASE) for p in config.generic_message_patterns)

    is_generic_message = combined_text.map(generic_hit)

    # Heuristic: low distinct token ratio (signals vague content)
    def distinct_token_ratio(text: str) -> float:
        tokens = re.findall(r"\b\w+\b", (text or "").lower(), flags=re.UNICODE)
        if len(tokens) == 0:
            return 0.0
        return len(set(tokens)) / len(tokens)

    distinct_ratio = combined_text.map(distinct_token_ratio)
    is_low_distinct_ratio = distinct_ratio < 0.45  # heuristic threshold

    is_low_information = is_very_short | (is_generic_message & is_low_distinct_ratio)

    summary = {
        "very_short_row_count": int(is_very_short.sum()),
        "very_short_ratio": float(is_very_short.mean()),
        "generic_message_row_count": int(is_generic_message.sum()),
        "low_distinct_ratio_row_count": int(is_low_distinct_ratio.sum()),
        "low_information_row_count": int(is_low_information.sum()),
        "low_information_ratio": float(is_low_information.mean()),
        "thresholds": {
            "minimum_character_count": int(config.minimum_character_count),
            "minimum_word_count": int(config.minimum_word_count),
            "low_distinct_ratio_threshold": 0.45,
        },
    }

    artifacts = {
        "character_count": character_count,
        "word_count": word_count,
        "distinct_token_ratio": distinct_ratio,
    }

    masks = {
        "is_very_short": is_very_short,
        "is_generic_message": is_generic_message,
        "is_low_distinct_ratio": is_low_distinct_ratio,
        "is_low_information": is_low_information,
    }

    return {"summary": summary, "masks": masks, "artifacts": artifacts}

In [9]:
# 3.3 Duplicidade de tickets (exata e quase duplicada)

def check_duplicate_tickets(combined_text: pd.Series, config: ticket_dataset_quality_config) -> Dict[str, Any]:
    normalized = combined_text.fillna("").astype(str).map(normalize_text)

    is_exact_duplicate = normalized.duplicated(keep=False) & (normalized.str.len() > 0)
    exact_duplicate_row_count = int(is_exact_duplicate.sum())
    exact_duplicate_group_count = int(normalized[is_exact_duplicate].nunique())

    # Near duplicates: TF-IDF cosine similarity (sample capped)
    non_empty = normalized[normalized.str.len() > 0]
    if non_empty.shape[0] < 10:
        near_duplicate_summary = {
            "method": "tfidf_cosine_similarity",
            "evaluated_row_count": int(non_empty.shape[0]),
            "near_duplicate_pair_count": 0,
            "near_duplicate_row_count": 0,
            "note": "Not enough non-empty rows to evaluate near duplicates.",
        }
    else:
        if non_empty.shape[0] > config.near_duplicate_sample_size:
            sampled = non_empty.sample(config.near_duplicate_sample_size, random_state=42)
        else:
            sampled = non_empty

        vectorizer = TfidfVectorizer(
            max_features=config.tfidf_max_features,
            min_df=config.tfidf_min_document_frequency,
            max_df=config.tfidf_max_document_frequency,
            ngram_range=(1, 2),
        )
        tfidf = vectorizer.fit_transform(sampled.tolist())
        similarity = cosine_similarity(tfidf)

        upper = np.triu_indices(similarity.shape[0], k=1)
        similarities = similarity[upper]
        near_duplicate_pair_count = int(np.sum(similarities >= config.near_duplicate_similarity_threshold))

        adjacency = similarity >= config.near_duplicate_similarity_threshold
        np.fill_diagonal(adjacency, False)
        near_duplicate_row_count = int(np.sum(adjacency.any(axis=1)))

        near_duplicate_summary = {
            "method": "tfidf_cosine_similarity",
            "evaluated_row_count": int(sampled.shape[0]),
            "near_duplicate_pair_count": near_duplicate_pair_count,
            "near_duplicate_row_count": near_duplicate_row_count,
            "similarity_threshold": float(config.near_duplicate_similarity_threshold),
            "note": "Counts are based on a sample if dataset is large.",
        }

    summary = {
        "exact_duplicate_row_count": exact_duplicate_row_count,
        "exact_duplicate_group_count": exact_duplicate_group_count,
        "near_duplicates": near_duplicate_summary,
    }

    masks = {
        "is_exact_duplicate": is_exact_duplicate,
    }

    return {"summary": summary, "masks": masks}

In [10]:
# 3.4 Presença de ruído textual (assinaturas, disclaimers e threads)

def detect_text_noise_patterns(base: pd.DataFrame, combined_text: pd.Series, config: ticket_dataset_quality_config) -> Dict[str, Any]:
    # Thread indicators
    thread_line_patterns = [
        r"^\s*de:\s", r"^\s*from:\s",
        r"^\s*para:\s", r"^\s*to:\s",
        r"^\s*assunto:\s", r"^\s*subject:\s",
        r"^\s*enviado em:\s", r"^\s*sent:\s",
        r"^\s*data:\s", r"^\s*date:\s",
        r"^\s*cc:\s",
        r"^\s*---+\s*original message\s*---+",
        r"^\s*---+\s*mensagem original\s*---+",
    ]

    signature_patterns = [
        r"\batenciosamente\b",
        r"\batt\.?\b",
        r"\babra(ç|c)o(s)?\b",
        r"\bobrigad[oa]\b",
        r"\bsent from my (iphone|android)\b",
        r"\benviado do meu (iphone|android)\b",
        r"\bwhatsapp\b",
        r"\btelefone\b",
        r"\bcel(ular)?\b",
    ]

    disclaimer_patterns = [
        r"\besta mensagem\b.*\bconfidencial\b",
        r"\bconfidentiality\b",
        r"\bse você recebeu esta mensagem por engano\b",
        r"\bpor favor, apague\b",
    ]

    def extract_noise_flags(text: str) -> Dict[str, Any]:
        text = text or ""
        lines = text.split("\n")[: config.maximum_lines_to_consider]
        text_lower = text.lower()

        thread_hits = 0
        for line in lines:
            line_lower = line.lower()
            for p in thread_line_patterns:
                if re.search(p, line_lower, flags=re.IGNORECASE):
                    thread_hits += 1
                    break

        signature_hits = sum(len(re.findall(p, text_lower, flags=re.IGNORECASE)) for p in signature_patterns)
        disclaimer_hits = sum(len(re.findall(p, text_lower, flags=re.IGNORECASE | re.DOTALL)) for p in disclaimer_patterns)

        return {
            "thread_indicator_count": int(thread_hits),
            "signature_indicator_count": int(signature_hits),
            "disclaimer_indicator_count": int(disclaimer_hits),
            "has_thread_markers": bool(thread_hits >= 1),
            "has_signature_markers": bool(signature_hits >= 2),
            "has_disclaimer_markers": bool(disclaimer_hits >= 1),
        }

    noise_features_series = combined_text.map(extract_noise_flags)
    noise_features_dataframe = pd.DataFrame(noise_features_series.tolist(), index=base.index)

    summary = {
        "thread_markers_row_count": int(noise_features_dataframe["has_thread_markers"].sum()),
        "signature_markers_row_count": int(noise_features_dataframe["has_signature_markers"].sum()),
        "disclaimer_markers_row_count": int(noise_features_dataframe["has_disclaimer_markers"].sum()),
        "thread_markers_row_ratio": float(noise_features_dataframe["has_thread_markers"].mean()),
        "signature_markers_row_ratio": float(noise_features_dataframe["has_signature_markers"].mean()),
        "disclaimer_markers_row_ratio": float(noise_features_dataframe["has_disclaimer_markers"].mean()),
    }

    return {"summary": summary, "noise_features_dataframe": noise_features_dataframe}


In [11]:
# 3.5 Distribuição do tamanho do texto

def compute_text_length_distribution(combined_text: pd.Series) -> Dict[str, Any]:
    character_count = combined_text.fillna("").astype(str).map(len)
    word_count = combined_text.fillna("").astype(str).map(count_words)

    def describe_numeric(values: pd.Series) -> Dict[str, float]:
        return {
            "count": float(values.shape[0]),
            "mean": float(values.mean()),
            "median": float(values.median()),
            "p90": float(values.quantile(0.90)),
            "p95": float(values.quantile(0.95)),
            "p99": float(values.quantile(0.99)),
            "max": float(values.max()),
            "min": float(values.min()),
        }

    summary = {
        "character_count_summary": describe_numeric(character_count),
        "word_count_summary": describe_numeric(word_count),
    }

    artifacts = {"character_count": character_count, "word_count": word_count}
    return {"summary": summary, "artifacts": artifacts}



In [12]:
# 3.6 Síntese da análise final do dataset
# ============================================================
def stage_3_6_build_dataset_xray(
    base: pd.DataFrame,
    combined_text: pd.Series,
    results_3_1: Dict[str, Any],
    results_3_2: Dict[str, Any],
    results_3_3: Dict[str, Any],
    results_3_4: Dict[str, Any],
    results_3_5: Dict[str, Any],
) -> Dict[str, Any]:
    # Consolidate risk flags in one dataframe for inspection and downstream pipeline decisions.
    noise_df = results_3_4["noise_features_dataframe"]

    risk_flags_dataframe = pd.DataFrame(index=base.index)
    for key, mask in results_3_1["masks"].items():
        risk_flags_dataframe[key] = mask
    for key, mask in results_3_2["masks"].items():
        risk_flags_dataframe[key] = mask
    for key, mask in results_3_3["masks"].items():
        risk_flags_dataframe[key] = mask

    risk_flags_dataframe = risk_flags_dataframe.join(noise_df)
    risk_flags_dataframe["character_count"] = results_3_5["artifacts"]["character_count"]
    risk_flags_dataframe["word_count"] = results_3_5["artifacts"]["word_count"]

    # High risk definition: low info OR heavy noise OR duplicates OR both empty
    is_high_risk_for_classification = (
        risk_flags_dataframe["is_both_empty"]
        | risk_flags_dataframe["is_low_information"]
        | risk_flags_dataframe["is_exact_duplicate"]
        | risk_flags_dataframe["has_thread_markers"]
        | risk_flags_dataframe["has_disclaimer_markers"]
    )
    risk_flags_dataframe["is_high_risk_for_classification"] = is_high_risk_for_classification

    xray_summary = {
        "dataset_shape": {"row_count": int(base.shape[0]), "column_count": int(base.shape[1])},
        "high_risk_row_count": int(is_high_risk_for_classification.sum()),
        "high_risk_ratio": float(is_high_risk_for_classification.mean()),
        "stage_3_1_missing_fields": results_3_1["summary"],
        "stage_3_2_short_low_info": results_3_2["summary"],
        "stage_3_3_duplicates": results_3_3["summary"],
        "stage_3_4_noise": results_3_4["summary"],
        "stage_3_6_length_distribution": results_3_5["summary"],
    }

    return {"summary": xray_summary, "risk_flags_dataframe": risk_flags_dataframe}


In [13]:
# Orchestrator: runs all sub-steps of Stage 3


def profile_ticket_dataset_quality(base: pd.DataFrame, config: Optional[ticket_dataset_quality_config] = None) -> Dict[str, Any]:
    if config is None:
        config = ticket_dataset_quality_config()

    required_columns = {config.subject_column_name, config.body_column_name}
    missing_columns = required_columns - set(base.columns)
    if missing_columns:
        raise ValueError(
            f"Missing required columns: {missing_columns}. "
            f"Expected columns: {config.subject_column_name!r}, {config.body_column_name!r}."
        )

    combined_text = build_combined_text(base, config)

    results_3_1 = check_missing_ticket_fields(base, config)
    results_3_2 = check_low_information_tickets(base, combined_text, config)
    results_3_3 = check_duplicate_tickets(combined_text, config)
    results_3_4 = detect_text_noise_patterns(base, combined_text, config)
    results_3_5 = compute_text_length_distribution(combined_text)
    results_3_6 = stage_3_6_build_dataset_xray(
        base=base,
        combined_text=combined_text,
        results_3_1=results_3_1,
        results_3_2=results_3_2,
        results_3_3=results_3_3,
        results_3_4=results_3_4,
        results_3_5=results_3_5
    )

    return {
        "combined_text": combined_text,
        "step_3_1": results_3_1,
        "step_3_2": results_3_2,
        "step_3_3": results_3_3,
        "step_3_4": results_3_4,
        "step_3_5": results_3_5,
        "step_3_6": results_3_6
    }

In [14]:
# Basic unit tests


def run_basic_unit_tests() -> None:
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer # Required for check_duplicate_tickets
    from sklearn.metrics.pairwise import cosine_similarity # Required for check_duplicate_tickets
    import numpy as np # Required for check_duplicate_tickets

    sample_dataframe = pd.DataFrame(
        {
            'subject': ['', 'Pagamento pendente', '', 'Oi', 'Oi'], # Two empty subjects for assertion, and duplicates for combined_text
            'body': ['Preciso de ajuda com reembolso', '', 'Preciso de ajuda com reembolso', 'ok', 'ok'], # One empty body for assertion, and duplicates for combined_text
        }
    )

    test_config = ticket_dataset_quality_config(
        subject_column_name='subject',
        body_column_name='body',
        minimum_character_count=10,
        minimum_word_count=2,
        near_duplicate_similarity_threshold=0.90
    )

    missing_report = check_missing_ticket_fields(sample_dataframe, test_config)
    assert missing_report['summary']['subject_empty_row_count'] == 2  # '' and None
    assert missing_report['summary']['body_empty_row_count'] == 1

    combined_text = build_combined_text(sample_dataframe, test_config)
    low_info_report = check_low_information_tickets(sample_dataframe, combined_text, test_config)
    assert low_info_report['summary']['very_short_row_count'] >= 1

    duplicate_report = check_duplicate_tickets(combined_text, test_config)
    assert duplicate_report['summary']['exact_duplicate_row_count'] >= 1

    noise_report = detect_text_noise_patterns(sample_dataframe, combined_text, test_config) # combined_text is now a required arg
    # Updated assertion to reflect the 'summary' structure
    assert 'signature_markers_row_count' in noise_report['summary']

    length_report = compute_text_length_distribution(combined_text)
    # Updated assertion to reflect the 'summary' structure
    assert length_report['summary']['character_count_summary']['min'] <= length_report['summary']['character_count_summary']['max']


run_basic_unit_tests()
print('basic_unit_tests_passed')

basic_unit_tests_passed


In [15]:
report = profile_ticket_dataset_quality(base)

In [16]:
print(report["step_3_1"]["summary"])
print(report["step_3_2"]["summary"])
print(report["step_3_3"]["summary"])
print(report["step_3_4"]["summary"])
print(report["step_3_5"]["summary"])
print(report["step_3_6"]["summary"])

{'subject_empty_row_count': 54, 'body_empty_row_count': 0, 'both_empty_row_count': 0, 'subject_empty_body_filled_row_count': 54, 'generic_subject_and_short_body_row_count': 0, 'subject_empty_ratio': 0.11416490486257928, 'body_empty_ratio': 0.0, 'both_empty_ratio': 0.0}
{'very_short_row_count': 4, 'very_short_ratio': 0.008456659619450317, 'generic_message_row_count': 219, 'low_distinct_ratio_row_count': 0, 'low_information_row_count': 4, 'low_information_ratio': 0.008456659619450317, 'thresholds': {'minimum_character_count': 60, 'minimum_word_count': 10, 'low_distinct_ratio_threshold': 0.45}}
{'exact_duplicate_row_count': 0, 'exact_duplicate_group_count': 0, 'near_duplicates': {'method': 'tfidf_cosine_similarity', 'evaluated_row_count': 473, 'near_duplicate_pair_count': 5, 'near_duplicate_row_count': 10, 'similarity_threshold': 0.92, 'note': 'Counts are based on a sample if dataset is large.'}}
{'thread_markers_row_count': 0, 'signature_markers_row_count': 293, 'disclaimer_markers_row_c

**Conclusão da análise do dataset:**

1 Campos vazios (assunto e corpo):
subject_empty_row_count = 54 → 11,4% sem assunto
body_empty_row_count = 0
both_empty_row_count = 0

O corpo do e-mail contém a informação principal.Não há perda total de informação em nenhum ticket.

-Impacto no modelo: Baixo risco, desde que:assunto e corpo sejam unificados e
templates e placeholders sejam removidos.

Ação 1 : usar combined_text

2 Conteúdo curto ou pouco informativo
very_short_row_count = 0
generic_message_row_count = 219 → 46,3%
low_information_row_count = 0

Não existem tickets “curtos demais”. Porém, quase metade contém linguagem genérica (“erro”, “problema”, “urgente”, etc.). Ou seja, pode ter conflito entre categorias e reduzir a confianã do modelo

Ação 2: pré-processamento de ruído


3 Duplicidade de tickets
Duplicatas exatas: 0
Near-duplicates:
near_duplicate_pair_count = 5
near_duplicate_row_count = 10

Poucos tickets duplicados. Risco pontual de vazamento se esses pares caírem em treino e validação distintos.

Ação 3: exclusão de pares quase idênticos do conjunto de validação


4 Ruído textual (assinaturas, rodapés, threads)
signature_markers_row_count = 293 → 61,9%
thread_markers_row_count = 0
disclaimer_markers_row_count = 0

O principal ruído do dataset são assinaturas, fechamentos formais, texto institucional repetido

Ação 4: emoção de assinaturas, placeholders, rodapés) - ponto crítico 62% contém esse ruído.

6 Distribuição do tamanho do texto

Mediana: 730 caracteres / 106 palavras
p95: ~1700 caracteres / 252 palavras
Máximo: 2796 caracteres / 397 palavras

Dataset rico em conteúdo textual - sem problemas nesse quesito.

**Conclusão geral:**

A análise exploratória mostra que o dataset é adequado para classificação automática de tickets de TI, apresentando bom volume de informação textual e baixa duplicidade. Entretanto, são necessárias algumas ações para melhor desempenho do modelo.


## 4. Preparação do texto no dataset

Exclusão dos "erros" textuais para menor impacto no modelo final

In [17]:
#Concatena assunto e corpo com um separador claro - ajuda o modelo porque o assunto frequentemente contém palavras-chave.

subject_column_name = "subject"
body_column_name = "body"

base["combined_text"] = (
    base[subject_column_name].fillna("").astype(str).str.strip()
    + "\n"
    + base[body_column_name].fillna("").astype(str).str.strip()
).str.strip()


In [18]:
## REMOÇÃO DE RUÍDOS NO DATASET - MELHORIA DA PERFORMANCE DO MODELO

# 1 Remove formatação HTML
def remove_html_formatting(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = html.unescape(text)
    text = re.sub(r"<[^>]+>", " ", text)
    return text

# 2 Remove links e e-mails
def remove_links_and_email_addresses(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r"http[s]?://\S+|www\.\S+", " ", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", " ", text)
    return text

# 3 Remove placeholders tipo <name>, <tel_num>
def remove_placeholder_tokens(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r"<\s*[a-zA-Z_][a-zA-Z0-9_]*\s*>", " ", text)
    return text

# 4 Remove threads de e-mail (mensagens anteriores)
thread_marker_patterns = [
    r"(?im)^\s*de:\s",
    r"(?im)^\s*from:\s",
    r"(?im)^\s*para:\s",
    r"(?im)^\s*to:\s",
    r"(?im)^\s*assunto:\s",
    r"(?im)^\s*subject:\s",
    r"(?im)^\s*enviado em:\s",
    r"(?im)^\s*sent:\s",
    r"(?im)^\s*data:\s",
    r"(?im)^\s*---+\s*mensagem original\s*---+"
]

def remove_email_threads(text: str) -> str:
    if text is None:
        return ""
    text = str(text)

    earliest_cut_position = None
    for pattern in thread_marker_patterns:
        match = re.search(pattern, text)
        if match:
            earliest_cut_position = match.start() if earliest_cut_position is None else min(earliest_cut_position, match.start())

    if earliest_cut_position is not None and earliest_cut_position > 0:
        return text[:earliest_cut_position].strip()

    return text.strip()

# 5 Remove assinaturas, rodapés e disclaimers
footer_marker_patterns = [
    r"(?im)^\s*atenciosamente\b",
    r"(?im)^\s*att\.?\b",
    r"(?im)^\s*abra(ç|c)o(s)?\b",
    r"(?im)^\s*obrigad[oa]\b",
    r"(?im)^\s*cordialmente\b",
    r"(?im)^\s*enviado do meu (iphone|android)\b",
    r"(?im)\besta mensagem\b.*\bconfidencial\b",
    r"(?im)\bse você recebeu esta mensagem\b",
]

def remove_signatures_and_disclaimers(text: str) -> str:
    if text is None:
        return ""
    text = str(text)

    earliest_cut_position = None
    for pattern in footer_marker_patterns:
        match = re.search(pattern, text)
        if match:
            earliest_cut_position = match.start() if earliest_cut_position is None else min(earliest_cut_position, match.start())

    if earliest_cut_position is not None and earliest_cut_position > 0:
        return text[:earliest_cut_position].strip()

    return text.strip()

# 6 Remove IDs irrelevantes
def remove_irrelevant_identifiers(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = re.sub(
        r"\b[a-f0-9]{8}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{12}\b",
        " ",
        text,
        flags=re.IGNORECASE,
    )
    text = re.sub(r"\b[a-zA-Z0-9]{25,}\b", " ", text)
    return text

# 7 Normalização final
def normalize_text_for_modeling(text: str) -> str:
    if text is None:
        return ""
    text = str(text).lower()
    text = re.sub(r"([!?\.]){2,}", r"\1", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [19]:
# APPLY PIPELINE (dataset final pronto para usar)

base["combined_text_without_html"] = base["combined_text"].map(remove_html_formatting)
base["combined_text_without_links"] = base["combined_text_without_html"].map(remove_links_and_email_addresses)
base["combined_text_without_placeholders"] = base["combined_text_without_links"].map(remove_placeholder_tokens)

# Opcional mas recomendado:
base["combined_text_without_threads"] = base["combined_text_without_placeholders"].map(remove_email_threads)


base["combined_text_without_footer"] = base["combined_text_without_threads"].map(remove_signatures_and_disclaimers)
base["combined_text_without_identifiers"] = base["combined_text_without_footer"].map(remove_irrelevant_identifiers)
base["clean_text"] = base["combined_text_without_identifiers"].map(normalize_text_for_modeling)

# Dataset final mínimo para modelagem
final_dataset = base[["clean_text"]].copy()

In [20]:
final_dataset.count()

,0
clean_text,473


In [21]:
final_dataset.head(50)

,clean_text
0,erro na autocompletação de código do intellij ...
1,problemas intermitentes de exibição após atual...
2,pedido de assistência para administração de se...
3,"prezado , sentimos muito ao saber sobre o atra..."
4,urgente: relato de falha na bateria do macbook...
5,assistência necessária para integração do jira...
6,discrepância de faturamento no google workspac...
7,urgente: problema de inatividade do serviço de...
8,solicitação de soluções de gerenciamento da aw...
9,interrupção do roteador cisco caro suporte ao ...


In [22]:
def clean_ticket_text_for_modeling(text: str) -> str:
    text = remove_html_formatting(text)
    text = remove_links_and_email_addresses(text)
    text = remove_placeholder_tokens(text)
    text = remove_email_threads(text)
    text = remove_signatures_and_disclaimers(text)
    text = remove_irrelevant_identifiers(text)
    text = normalize_text_for_modeling(text)
    return text


# Teste de integração
test_texts = [
    None,
    "<b>Erro</b> no sistema!!!",
    "Acesse https://example.com para mais info",
    "Olá <name>, preciso de ajuda",
    "Mensagem atual\n\nDe: Fulano\nAssunto: erro",
    "Texto principal\n\nAtenciosamente,\nEquipe",
    "ID 123e4567-e89b-12d3-a456-426614174000 erro crítico!!!",
    "   TEXTO   COM   MUITOS   ESPAÇOS!!!   ",
]

cleaned_results = [clean_ticket_text_for_modeling(raw_text) for raw_text in test_texts]

for result in cleaned_results:
    assert isinstance(result, str)
    assert "<" not in result and ">" not in result
    assert "http" not in result and "www." not in result
    assert "@" not in result
    assert "  " not in result
    assert result == result.lower()
    assert re.search(
        r"[a-f0-9]{8}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{12}",
        result,
        flags=re.IGNORECASE,
    ) is None


In [23]:
list(zip(test_texts, cleaned_results))

#funções validadas


[(None, ''),
 ('<b>Erro</b> no sistema!!!', 'erro no sistema!'),
 ('Acesse https://example.com para mais info', 'acesse para mais info'),
 ('Olá <name>, preciso de ajuda', 'olá , preciso de ajuda'),
 ('Mensagem atual\n\nDe: Fulano\nAssunto: erro', 'mensagem atual'),
 ('Texto principal\n\nAtenciosamente,\nEquipe', 'texto principal'),
 ('ID 123e4567-e89b-12d3-a456-426614174000 erro crítico!!!',
  'id erro crítico!'),
 ('   TEXTO   COM   MUITOS   ESPAÇOS!!!   ', 'texto com muitos espaços!')]

obs: As funções originalmente desenvolvidas no Item 3 tinham como objetivo realizar o diagnóstico de qualidade dos textos e identificar padrões de ruído, campos ausentes e problemas de consistência no dataset. No Item 4, essas funções foram refatoradas e parcialmente reescritas com foco em aplicação prática no pipeline de modelagem, priorizando reutilização, clareza semântica e eficiência operacional. A refatoração permitiu consolidar lógicas duplicadas, padronizar nomes e responsabilidades das funções, e transformar análises diagnósticas em etapas efetivas de tratamento e preparação dos dados, reduzindo ruídos que impactariam diretamente a performance do modelo.

# 5. Construção do Modelo

**Estrutura do problema**

Cada ticket terá dois rótulos:
main_category

-connectivity
-computer_hardware
-software_operating_system
-access_permissions_accounts
-other

criticality

-urgent
-normal

In [24]:
# Dicionários de palavras-chave para rotular os tickts

CATEGORY_KEYWORDS = {
    "connectivity": [
        # conectividade
        "internet", "conexão", "conectividade", "rede",
        "wifi", "wi-fi", "wireless", "lan", "wan",

        # acesso remoto
        "vpn", "acesso remoto", "remote access",

        # sintomas
        "instável", "queda de conexão", "sem internet",
        "não conecta", "offline", "timeout", "latência",
        "lento na rede", "sem sinal",

        # infraestrutura
        "proxy", "firewall", "dns", "ip", "gateway",
    ],

    "computer_hardware": [
        # dispositivos
        "notebook", "laptop", "desktop", "computador",
        "pc", "máquina", "equipamento",

        # componentes
        "bateria", "carregador", "fonte", "tela",
        "monitor", "teclado", "mouse", "touchpad",
        "hd", "ssd", "memória", "ram", "processador",

        # sintomas físicos
        "não liga", "desligando sozinho", "superaquecendo",
        "barulho", "aquecendo", "travando",

        # performance ligada a hardware
        "muito lento", "performance baixa", "hardware",
    ],

    "software_operating_system": [
        # sistemas operacionais
        "windows", "linux", "ubuntu", "mac", "macos",

        # aplicações genéricas
        "software", "aplicação", "programa", "sistema",
        "app", "ferramenta",

        # IDEs e ferramentas comuns
        "vscode", "visual studio code", "intellij",
        "pycharm", "eclipse",

        # erros e falhas
        "erro", "bug", "falha", "crash",
        "não abre", "parou de funcionar",

        # atualizações
        "atualização", "update", "patch",
        "versão incompatível",
    ],

    "access_permissions_accounts": [
        # autenticação
        "login", "logar", "senha", "credencial",
        "autenticação", "autorização",

        # autorização
        "permissão", "permissão negada", "sem permissão",
        "acesso negado", "não autorizado",

        # conta
        "conta", "usuário", "perfil", "role",
        "bloqueado", "desbloqueio",

        # sistemas corporativos
        "single sign-on", "sso", "active directory",
        "ldap",
    ]
}


CRITICALITY_KEYWORDS = {
    "urgent": [
        "urgente", "critico", "imediato", "bloqueado", "parado",
        "não consigo trabalhar", "impacto crítico"
    ]
}


In [25]:
# rotulagem baseada no dicionário

def label_main_category(text: str) -> str:
    text_lower = text.lower()
    for category, keywords in CATEGORY_KEYWORDS.items():
        if any(keyword in text_lower for keyword in keywords):
            return category
    return "other"


def label_criticality(text: str) -> str:
    text_lower = text.lower()
    if any(keyword in text_lower for keyword in CRITICALITY_KEYWORDS["urgent"]):
        return "urgent"
    return "normal"


In [26]:
# aplicando a função de rotulagem inicial

final_dataset["main_category"] = final_dataset["clean_text"].apply(label_main_category)
final_dataset["criticality"] = final_dataset["clean_text"].apply(label_criticality)

In [27]:
final_dataset.head()

,clean_text,main_category,criticality
0,erro na autocompletação de código do intellij ...,software_operating_system,normal
1,problemas intermitentes de exibição após atual...,computer_hardware,normal
2,pedido de assistência para administração de se...,connectivity,normal
3,"prezado , sentimos muito ao saber sobre o atra...",computer_hardware,normal
4,urgente: relato de falha na bateria do macbook...,computer_hardware,urgent


A pré-rotulagem dos tickets foi realizada por meio de um dicionário de palavras-chave definido por categoria, mapeando termos e expressões recorrentes associados a problemas de rede, hardware, software/sistema operacional, acesso/permissões e uma classe de escape (outros), além de uma camada adicional de criticidade (urgent vs normal). Essa abordagem heurística permitiu gerar rótulos iniciais de forma automática e escalável, explorando padrões léxicos frequentes nos textos dos chamados e viabilizando a criação de um conjunto de treinamento inicial sem dependência imediata de rotulagem manual. Os rótulos gerados não são tratados como verdade absoluta, mas como weak labels, utilizados para bootstrap do modelo supervisionado, reduzindo esforço operacional e preparando o pipeline para refinamentos posteriores via validação humana, aprendizado semi-supervisionado e active learning.

**Implementação do modelo de classificação** - main class

In [28]:
# SPLIT GLOBAL ÚNICO (dataset completo, inclui "other")
# Estratificado por criticidade para garantir proporção urgent/normal
# obs.: antes eu tinha feito um split para o modelo de classes e um para a criticidade, mas depois decidi unificar

full_dataset = final_dataset.copy()

X_text_full = full_dataset["clean_text"].astype(str)
y_criticality_full = full_dataset["criticality"].astype(str)
y_category_full = full_dataset["main_category"].astype(str)

X_train_full, X_test_full, y_criticality_train, y_criticality_test = train_test_split(
    X_text_full,
    y_criticality_full,
    test_size=0.20,
    random_state=42,
    stratify=y_criticality_full
)

print(f"Global split | Train: {len(X_train_full)} | Test: {len(X_test_full)}")
print("Criticality distribution (train):")
print(y_criticality_train.value_counts(normalize=True))

Global split | Train: 378 | Test: 95
Criticality distribution (train):
criticality
normal    0.693122
urgent    0.306878
Name: proportion, dtype: float64


In [29]:
# VETORIZAÇÃO ÚNICA
# Compartilhada por ambos os modelos: reduz inconsistência e evita vazamento

word_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.90,
    sublinear_tf=True,
    norm="l2",
)

char_vectorizer = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=3,
    max_df=0.90,
    sublinear_tf=True,
    norm="l2",
)

X_train_word_full = word_vectorizer.fit_transform(X_train_full)
X_test_word_full = word_vectorizer.transform(X_test_full)

X_train_char_full = char_vectorizer.fit_transform(X_train_full)
X_test_char_full = char_vectorizer.transform(X_test_full)

X_train_full_final = hstack([X_train_word_full, X_train_char_full])
X_test_full_final = hstack([X_test_word_full, X_test_char_full])


Os textos foram vetorizados por meio da técnica TF-IDF, combinando n-gramas de palavras (unigramas e bigramas) e n-gramas de caracteres, com o ajuste realizado exclusivamente sobre o conjunto de treinamento para evitar vazamento de dados. A aplicação de limites de frequência mínima e máxima (min_df e max_df) permitiu reduzir ruído e termos pouco informativos, enquanto os n-gramas de caracteres aumentaram a robustez do modelo a variações ortográficas, abreviações e erros de digitação comuns em tickets de suporte. Essa estratégia resultou em um espaço vetorial esparso, porém discriminativo, adequado para modelos lineares de alta dimensionalidade, favorecendo a separação semântica entre categorias e a generalização para textos não vistos.

**Justificativa de escolha do modelo:**

O modelo LinearSVC foi escolhido por sua adequação a problemas de classificação de texto em espaços TF-IDF de alta dimensionalidade e natureza esparsa, como é o caso de tickets de suporte. Por ser um classificador linear baseado em máxima margem, o LinearSVC é capaz de aprender fronteiras de decisão robustas mesmo na presença de classes desbalanceadas e rótulos ruidosos, apresentando desempenho consistente em termos de F1 macro quando comparado a alternativas como Random Forest ou modelos não lineares. Além disso, sua eficiência computacional e estabilidade em conjuntos com grande número de features o tornam uma escolha apropriada para cenários produtivos, nos quais se busca bom poder discriminativo, escalabilidade e interpretabilidade dos pesos associados aos termos mais relevantes para cada classe.

**Modelo de criticidade**

In [30]:
# MODELO 1: CRITICIDADE (treina no dataset completo)
#

criticality_model = LinearSVC(class_weight="balanced")
criticality_model.fit(X_train_full_final, y_criticality_train)

y_criticality_test_pred = criticality_model.predict(X_test_full_final)

print("\n[CRITICALITY] Test accuracy:", accuracy_score(y_criticality_test, y_criticality_test_pred))
print("[CRITICALITY] Test f1_macro:", f1_score(y_criticality_test, y_criticality_test_pred, average="macro"))
print("[CRITICALITY] Test f1_urgent:", f1_score(y_criticality_test, y_criticality_test_pred, pos_label="urgent"))

print("\n[CRITICALITY] Classification report (test):")
print(classification_report(y_criticality_test, y_criticality_test_pred, digits=4))

print("\n[CRITICALITY] Confusion matrix (test) [urgent, normal]:")
print(confusion_matrix(y_criticality_test, y_criticality_test_pred, labels=["urgent", "normal"]))



[CRITICALITY] Test accuracy: 0.9052631578947369
[CRITICALITY] Test f1_macro: 0.8872180451127819
[CRITICALITY] Test f1_urgent: 0.8421052631578947

[CRITICALITY] Classification report (test):
              precision    recall  f1-score   support

      normal     0.9254    0.9394    0.9323        66
      urgent     0.8571    0.8276    0.8421        29

    accuracy                         0.9053        95
   macro avg     0.8913    0.8835    0.8872        95
weighted avg     0.9045    0.9053    0.9048        95


[CRITICALITY] Confusion matrix (test) [urgent, normal]:
[[24  5]
 [ 4 62]]


In [31]:
# Overfitting check (criticality)
y_criticality_train_pred = criticality_model.predict(X_train_full_final)

train_accuracy = accuracy_score(y_criticality_train, y_criticality_train_pred)
test_accuracy = accuracy_score(y_criticality_test, y_criticality_test_pred)

train_f1_macro = f1_score(y_criticality_train, y_criticality_train_pred, average="macro")
test_f1_macro = f1_score(y_criticality_test, y_criticality_test_pred, average="macro")

train_f1_urgent = f1_score(y_criticality_train, y_criticality_train_pred, pos_label="urgent")
test_f1_urgent = f1_score(y_criticality_test, y_criticality_test_pred, pos_label="urgent")

print("\n[CRITICALITY] Overfitting check:")
print(f"Train accuracy: {train_accuracy:.4f} | Test accuracy: {test_accuracy:.4f} | Gap: {(train_accuracy - test_accuracy):.4f}")
print(f"Train f1_macro: {train_f1_macro:.4f} | Test f1_macro: {test_f1_macro:.4f} | Gap: {(train_f1_macro - test_f1_macro):.4f}")
print(f"Train f1_urgent: {train_f1_urgent:.4f} | Test f1_urgent: {test_f1_urgent:.4f} | Gap: {(train_f1_urgent - test_f1_urgent):.4f}")



[CRITICALITY] Overfitting check:
Train accuracy: 1.0000 | Test accuracy: 0.9053 | Gap: 0.0947
Train f1_macro: 1.0000 | Test f1_macro: 0.8872 | Gap: 0.1128
Train f1_urgent: 1.0000 | Test f1_urgent: 0.8421 | Gap: 0.1579


In [32]:
# Cross-validation (criticality) no treino global
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_accuracy_scores = []
cv_f1_macro_scores = []
cv_f1_urgent_scores = []

y_train_for_cv = y_criticality_train.reset_index(drop=True)

for train_idx, val_idx in cv.split(X_train_full_final, y_train_for_cv):
    X_tr = X_train_full_final[train_idx]
    X_val = X_train_full_final[val_idx]
    y_tr = y_train_for_cv.iloc[train_idx]
    y_val = y_train_for_cv.iloc[val_idx]

    fold_model = LinearSVC(class_weight="balanced")
    fold_model.fit(X_tr, y_tr)

    y_val_pred = fold_model.predict(X_val)

    cv_accuracy_scores.append(accuracy_score(y_val, y_val_pred))
    cv_f1_macro_scores.append(f1_score(y_val, y_val_pred, average="macro"))
    cv_f1_urgent_scores.append(f1_score(y_val, y_val_pred, pos_label="urgent"))

print("\n[CRITICALITY] CV results:")
print("CV accuracy mean:", float(np.mean(cv_accuracy_scores)), "std:", float(np.std(cv_accuracy_scores)))
print("CV f1_macro mean:", float(np.mean(cv_f1_macro_scores)), "std:", float(np.std(cv_f1_macro_scores)))
print("CV f1_urgent mean:", float(np.mean(cv_f1_urgent_scores)), "std:", float(np.std(cv_f1_urgent_scores)))




[CRITICALITY] CV results:
CV accuracy mean: 0.9152280701754387 std: 0.03739228296700755
CV f1_macro mean: 0.8978178903712205 std: 0.045400273827998404
CV f1_urgent mean: 0.8557175058708953 std: 0.06459021127676923


**Avaliação do modelo: **

O modelo de criticidade, conforme esperado, apresentou desempenho elevado e consistente no conjunto de teste, com acurácia de aproximadamente 90,5% e F1 macro de 0,89, indicando boa capacidade de discriminação entre chamados urgent e normal, mesmo diante do desbalanceamento natural entre as classes.

O F1 da classe urgente (≈ 0,84) demonstra que o modelo identifica de forma eficaz a maioria dos tickets críticos, mantendo bom equilíbrio entre precisão (≈ 0,86) e recall (≈ 0,83), o que é essencial para minimizar o risco operacional de falsos negativos e a boa identificação de tickets urgentes.

A matriz de confusão confirma esse comportamento, com a maior parte dos chamados urgentes corretamente classificados e um número limitado de erros, predominantemente confundindo urgentes com normais.

A análise de overfitting mostra gaps moderados entre treino e teste — esperados em modelos lineares treinados sobre representações TF-IDF de alta dimensionalidade — sem evidência de alto overfitting. Essa conclusão é reforçada pelos resultados de validação cruzada, cujas médias (CV F1 macro ≈ 0,90 e CV F1 urgent ≈ 0,86) são consistentes com o desempenho no teste e apresentam desvios padrão relativamente baixos, indicando estabilidade, boa generalização e robustez do modelo para uso em produção.


**Modelo de classes**

In [33]:
# MODELO 2: CATEGORIA (treina SEM 'other' e mantém o split global)
#
train_indices = X_train_full.index
test_indices = X_test_full.index

train_category_mask = (y_category_full.loc[train_indices] != "other")
test_category_mask = (y_category_full.loc[test_indices] != "other")

train_category_indices = train_indices[train_category_mask]
test_category_indices = test_indices[test_category_mask]

y_category_train = y_category_full.loc[train_category_indices]
y_category_test = y_category_full.loc[test_category_indices]

# mapear índices -> posições na matriz esparsa (posicional)
train_pos_map = pd.Series(np.arange(len(train_indices)), index=train_indices)
test_pos_map = pd.Series(np.arange(len(test_indices)), index=test_indices)

train_positions = train_pos_map.loc[train_category_indices].to_numpy()
test_positions = test_pos_map.loc[test_category_indices].to_numpy()

X_train_category_final = X_train_full_final[train_positions]
X_test_category_final = X_test_full_final[test_positions]

print("\n[CATEGORY] Derived split from global split:")
print(f"Train category: {X_train_category_final.shape[0]} | Test category: {X_test_category_final.shape[0]}")
print("Category distribution (train):")
print(y_category_train.value_counts(normalize=True))


category_model = LinearSVC(class_weight="balanced")
category_model.fit(X_train_category_final, y_category_train)

y_category_test_pred = category_model.predict(X_test_category_final)

print("\n[CATEGORY] Test accuracy:", accuracy_score(y_category_test, y_category_test_pred))
print("[CATEGORY] Test f1_macro:", f1_score(y_category_test, y_category_test_pred, average="macro"))

print("\n[CATEGORY] Classification report (test):")
print(classification_report(y_category_test, y_category_test_pred, digits=4))

print("\n[CATEGORY] Confusion matrix (test):")
print(confusion_matrix(y_category_test, y_category_test_pred))



[CATEGORY] Derived split from global split:
Train category: 367 | Test category: 91
Category distribution (train):
main_category
connectivity                   0.547684
computer_hardware              0.217984
software_operating_system      0.128065
access_permissions_accounts    0.106267
Name: proportion, dtype: float64

[CATEGORY] Test accuracy: 0.7582417582417582
[CATEGORY] Test f1_macro: 0.6062097902097902

[CATEGORY] Classification report (test):
                             precision    recall  f1-score   support

access_permissions_accounts     0.7500    0.4286    0.5455         7
          computer_hardware     0.6154    0.6154    0.6154        13
               connectivity     0.8182    0.9153    0.8640        59
  software_operating_system     0.5000    0.3333    0.4000        12

                   accuracy                         0.7582        91
                  macro avg     0.6709    0.5731    0.6062        91
               weighted avg     0.7420    0.7582    0.7428 

In [34]:
# Overfitting check (classe)
y_category_train_pred = category_model.predict(X_train_category_final)

train_accuracy = accuracy_score(y_category_train, y_category_train_pred)
test_accuracy = accuracy_score(y_category_test, y_category_test_pred)

train_f1_macro = f1_score(y_category_train, y_category_train_pred, average="macro")
test_f1_macro = f1_score(y_category_test, y_category_test_pred, average="macro")

print("\n[CATEGORY] Overfitting check:")
print(f"Train accuracy: {train_accuracy:.4f} | Test accuracy: {test_accuracy:.4f} | Gap: {(train_accuracy - test_accuracy):.4f}")
print(f"Train f1_macro: {train_f1_macro:.4f} | Test f1_macro: {test_f1_macro:.4f} | Gap: {(train_f1_macro - test_f1_macro):.4f}")




[CATEGORY] Overfitting check:
Train accuracy: 1.0000 | Test accuracy: 0.7582 | Gap: 0.2418
Train f1_macro: 1.0000 | Test f1_macro: 0.6062 | Gap: 0.3938


In [35]:
# ---- Cross-validation (category) no treino de categoria
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_accuracy_scores = []
cv_f1_macro_scores = []

y_category_train_for_cv = y_category_train.reset_index(drop=True)

for train_idx, val_idx in cv.split(X_train_category_final, y_category_train_for_cv):
    X_tr = X_train_category_final[train_idx]
    X_val = X_train_category_final[val_idx]
    y_tr = y_category_train_for_cv.iloc[train_idx]
    y_val = y_category_train_for_cv.iloc[val_idx]

    fold_model = LinearSVC(class_weight="balanced")
    fold_model.fit(X_tr, y_tr)

    y_val_pred = fold_model.predict(X_val)

    cv_accuracy_scores.append(accuracy_score(y_val, y_val_pred))
    cv_f1_macro_scores.append(f1_score(y_val, y_val_pred, average="macro"))

print("\n[CATEGORY] CV results:")
print("CV accuracy mean:", float(np.mean(cv_accuracy_scores)), "std:", float(np.std(cv_accuracy_scores)))
print("CV f1_macro mean:", float(np.mean(cv_f1_macro_scores)), "std:", float(np.std(cv_f1_macro_scores)))


[CATEGORY] CV results:
CV accuracy mean: 0.7167345427619399 std: 0.03173233717022916
CV f1_macro mean: 0.5765130671544116 std: 0.05365732328932599


**Avaliação do modelo de classes**

O modelo de classificação de categorias apresentou desempenho sólido no conjunto de teste, com acurácia de aproximadamente 75,8% e F1 macro de 0,61, refletindo uma capacidade consistente de discriminação entre as quatro classes principais, mesmo diante de um cenário claramente desbalanceado.

A classe nconnectivity representa mais de 54% dos exemplos de treino.O bom desempenho dessa classe é evidenciado por alto recall (≈ 0,92) e F1-score elevado (≈ 0,86), indicando que o modelo identifica corretamente a maioria dos tickets relacionados à rede.

As classes computer_hardware e access_permissions_accounts apresentaram desempenho intermediário, com precisão e recall moderados, enquanto software_operating_system mostrou maior dificuldade, especialmente em recall, refletindo a sobreposição semântica com outras categorias e o menor volume de exemplos disponíveis.

A matriz de confusão confirma que os principais erros decorrem da confusão entre software_operating_system, computer_hardware e connectivity, um padrão esperado em textos de suporte técnico.

A análise de overfitting revela um gap elevado entre treino e teste — comum em modelos lineares treinados sobre representações TF-IDF de alta dimensionalidade e rótulos fracos — porém a validação cruzada apresenta métricas médias (CV F1 macro ≈ 0,58) próximas às obtidas no teste, com baixo desvio padrão, indicando estabilidade e boa capacidade de generalização dentro das limitações impostas pelo volume e qualidade dos dados.



In [36]:
# Importância das variáveis


def get_feature_names_with_prefix(word_vectorizer, char_vectorizer):
    """
    Builds a unified feature name list matching the hstack([word_tfidf, char_tfidf]) order.
    Adds prefixes so you can distinguish word vs char features.
    """
    word_feature_names = word_vectorizer.get_feature_names_out()
    char_feature_names = char_vectorizer.get_feature_names_out()

    word_feature_names = np.array([f"word::{name}" for name in word_feature_names], dtype=object)
    char_feature_names = np.array([f"char::{name}" for name in char_feature_names], dtype=object)

    all_feature_names = np.concatenate([word_feature_names, char_feature_names])
    return all_feature_names


def print_top_features_per_class(linear_svc_model, feature_names, top_n=20):
    """
    Prints the top_n features with the highest positive weights for each class.
    For multiclass LinearSVC (one-vs-rest), coef_ has shape [n_classes, n_features].
    """
    class_labels = linear_svc_model.classes_
    coef_matrix = linear_svc_model.coef_  # shape: (n_classes, n_features)

    for class_index, class_label in enumerate(class_labels):
        class_coef = coef_matrix[class_index]

        # Top positive weights (features most indicative of this class)
        top_indices = np.argsort(class_coef)[-top_n:][::-1]
        top_features = feature_names[top_indices]
        top_weights = class_coef[top_indices]

        print(f"\nTop {top_n} features for class '{class_label}':")
        for feat, w in zip(top_features, top_weights):
            print(f"  {feat:<40} {w:>10.4f}")


# Build feature names aligned with your hstack order
all_feature_names = get_feature_names_with_prefix(word_vectorizer, char_vectorizer)

# Print per-class top features
print_top_features_per_class(category_model, all_feature_names, top_n=20)



Top 20 features for class 'access_permissions_accounts':
  word::administração do                       0.4460
  word::troca para                             0.4351
  word::de papel                               0.4253
  word::papel                                  0.4194
  word::de administração                       0.4163
  word::administração                          0.4092
  word::detectado                              0.4024
  word::de cobrança                            0.3895
  word::compra                                 0.3852
  word::da aws                                 0.3690
  word::atolamentos                            0.3599
  word::atolamentos de                         0.3599
  word::sob                                    0.3510
  word::cobrança                               0.3440
  word::parece                                 0.3436
  word::serviços em                            0.3414
  word::unidade                                0.3333
  word::aws caro        

A análise dos coeficientes do modelo LinearSVC evidencia que o classificador aprendeu padrões léxicos semanticamente coerentes com cada categoria, reforçando a interpretabilidade da solução.

Para access_permissions_accounts, destacam-se termos relacionados a administração, cobrança, serviços em nuvem e permissões de sistemas corporativos (por exemplo, administração, AWS, cobrança, Slack), indicando que o modelo capturou corretamente o vocabulário típico de solicitações envolvendo acesso, gestão e autorizações.

Na classe computer_hardware, as features mais relevantes estão fortemente associadas a componentes físicos (tela, bateria, teclado, RAM), além de n-gramas de caracteres que aumentam a robustez a variações ortográficas, confirmando uma separação clara baseada em falhas ou limitações de equipamentos.

A classe connectivity apresenta pesos elevados para termos ligados a rede, conectividade e infraestrutura (rede, conectividade, LAN, equipe), com destaque para n-gramas de caracteres que capturam variações linguísticas comuns nesses relatos.

Por fim, em software_operating_system, predominam expressões associadas a instalação, problemas em aplicações e sistemas corporativos (instalação, sistema, Adobe, Office, Jira), refletindo adequadamente tickets relacionados a falhas ou configurações de software.

De forma geral, a distribuição das features confirma que a combinação de TF-IDF (palavras e caracteres) com um modelo linear permitiu aprender sinais discriminativos consistentes com o domínio do problema, validando tanto a qualidade dos dados quanto a adequação da abordagem adotada.

# 6. Predição de novos tickets

In [37]:
# predição de novos tickets

def predict_ticket(text: str, category_margin_threshold: float = 0.20) -> dict:
    text_series = pd.Series([str(text)])

    word_features = word_vectorizer.transform(text_series)
    char_features = char_vectorizer.transform(text_series)
    combined_features = hstack([word_features, char_features])

    predicted_criticality = str(criticality_model.predict(combined_features)[0])

    category_scores = category_model.decision_function(combined_features)
    best_margin = float(np.max(category_scores))
    predicted_category = str(category_model.predict(combined_features)[0])

    if best_margin < category_margin_threshold:
        predicted_category = "other"

    return {
        "main_category": predicted_category,
        "criticality": predicted_criticality,
        "category_margin": round(best_margin, 4)
    }

print(predict_ticket("não consigo acessar a vpn e estou bloqueado, urgente"))


{'main_category': 'other', 'criticality': 'urgent', 'category_margin': -0.0357}


In [38]:
# exemplo 1

# Exemplo 1 — Ticket de rede com alta criticidade
ticket_1 = (
    "Não consigo acessar a VPN desde esta manhã, a conexão cai o tempo todo "
    "e estou completamente bloqueado para trabalhar. Preciso de ajuda urgente."
)

print("Example 1 prediction:")
print(predict_ticket(ticket_1))

Example 1 prediction:
{'main_category': 'other', 'criticality': 'urgent', 'category_margin': 0.1249}


In [39]:
#exemplo 2

# Exemplo 2 — Ticket ambíguo / baixa confiança (fallback para 'other')
ticket_2 = (
    "Olá, gostaria de uma orientação sobre um procedimento interno que "
    "não encontrei na documentação e não sei qual sistema utilizar."
)

print("\nExample 2 prediction:")
print(predict_ticket(ticket_2))


Example 2 prediction:
{'main_category': 'other', 'criticality': 'normal', 'category_margin': -0.1752}


# 7. Resposta as perguntas

**1. Já que as classes foram definidas por você, como você avaliaria a classificação feita em novos
tickets?**

Como as classes iniciais foram definidas por mim via weak supervision (dicionário + regras), eu trataria a avaliação de novos tickets como um processo em duas camadas: primeiro, validaria a consistência interna do modelo com o esquema de classes (o que mede “aderência ao rótulo fraco”); em seguida, estabeleceria um mecanismo contínuo de validação com ground truth humano para medir a performance “real” do negócio.

Na prática, eu criaria um gold set incremental (amostragem recorrente de tickets recentes) priorizando casos de maior risco e maior incerteza: tickets com baixa margem (category_margin próxima ao limiar), tickets preditos como other, amostras das classes minoritárias (ex.: access_permissions_accounts e software_operating_system), e casos em que regras e modelo divergirem.

A métrica principal para categoria seria F1 macro (evita mascarar erro sob desbalanceamento), e para criticidade eu privilegiaria recall e F1 de urgent, porque o custo de falso negativo (classificar urgente como normal) tende a ser operacionalmente mais alto.

Também avaliaria a performance por faixas de confiança (ex.: F1 para margin > 0,2 / 0,4 / 0,6), pois isso permite transformar “confiança” em decisão de produto, ajustando o fallback other para maximizar segurança e cobertura.




**2. Como você iria monitorar essa solução? Que métricas você iria monitorar?**

Para monitorar a solução em produção, eu separaria o monitoramento em métricas de comportamento do modelo, qualidade do dado e impacto operacional, porque problemas diferentes se manifestam em camadas distintas.

No comportamento do modelo, monitoraria continuamente a distribuição das classes preditas (detecção de mudanças abruptas), a taxa de fallback para other (principal indicador de incerteza e drift), e a distribuição da margem do SVM (queda de p50/p95 de margem é um sinal precoce de drift semântico). Para criticidade, eu trataria recall de urgent como SLO quando houver feedback humano ou auditoria, e acompanharia a taxa de discordância entre urgência prevista e urgência corrigida por agentes.

Em qualidade de dado, eu acompanharia drift de texto (mudança de vocabulário, n-grams, comprimento médio, proporção de textos muito curtos, presença de templates e assinaturas), além de integridade (campos vazios, duplicatas e spikes de ruído).

Em impacto operacional, as métricas mais relevantes seriam taxa de reclassificação manual, tempo até primeira resposta e SLA por criticidade, assertividade de roteamento por equipe (quando a categoria alimenta filas), e efeitos em backlog (redução de retrabalho e redistribuição).

Com esses sinais, eu implementaria alertas simples: aumento sustentado de other, queda de margem média, mudança abrupta no mix de classes e aumento de correção manual — cada alerta com runbook de diagnóstico e ações (ex.: ajuste de dicionário, revisão de threshold, coleta de rótulos).



**3. Haveria retreino? Qual seria a estratégia para rodar o treino novamente? Qual periodicidade?**

Sim, haveria retreino, porque tickets mudam com o tempo (novos sistemas, novas políticas, mudanças de vocabulário e processos) e isso gera data drift e concept drift.

Eu adotaria uma estratégia híbrida: um retreino periódico mínimo (tipicamente mensal no início, podendo virar bimestral/trimestral se o ambiente for estável) e um retreino por gatilho, acionado quando métricas de drift e qualidade indicarem degradação (aumento de other, queda de margens, aumento de reclassificação manual, mudança de distribuição de classes, piora de recall em urgent).

O retreino usaria uma janela deslizante (por exemplo, últimos 3–6 meses) para capturar linguagem atual, com um buffer histórico para preservar padrões de classes raras, e manteria um conjunto fixo de validação para garantir comparabilidade entre versões. Do ponto de vista de MLOps, eu versionaria explicitamente: modelo, vetorizadores TF-IDF, threshold de fallback e métricas offline/online por versão, com rollout controlado e critério de rollback baseado em métricas de produção.

**4. (Bônus) Se decidíssemos rotular 1% desses dados agora, como você mudaria sua abordagem para um aprendizado semi-supervisionado ou active learning?**

Se decidirmos rotular 1% dos dados agora (imaginei que fosse de forma manual), eu mudaria a abordagem para maximizar retorno por rótulo, combinando active learning + semi-supervisionado.

Em vez de rotular 1% aleatoriamente (que tende a concentrar em redes e não melhora as classes difíceis), eu selecionaria amostras informativas: baixa margem (incerteza), tickets em other, divergências entre regra e modelo, e amostras que cubram diversidade semântica (clusterização para evitar redundância).

Esse 1% se torna um gold set de alta qualidade, que eu usaria para

(i) recalibrar/ajustar thresholds

(ii) treinar um modelo supervisionado mais fiel ao negócio, priorizando F1 macro e recall de urgent.

Em paralelo, eu aplicaria self-training: o modelo treinado no gold set pseudo-rotula o restante com um filtro de alta confiança (margem alta), e eu re-treinaria ponderando exemplos humanos com peso maior e pseudo-rótulos com peso menor, reduzindo a influência do ruído do weak labeling.

Essa estratégia melhora especialmente as classes minoritárias e reduz confusões semânticas, além de diminuir a dependência do dicionário ao longo do tempo, mantendo other como mecanismo de segurança para casos realmente ambíguos ou fora de escopo.

# 8. Perspectivas futuras



Em uma abordagem mais sofisticada, especialmente em cenários com maior volume de dados ou maior variabilidade semântica, o dicionário inicial de palavras-chave poderia ser substituído — ou complementado — por uma LLM atuando como rotulador.

Nesse desenho, a LLM seria utilizada para gerar rótulos iniciais de categoria e criticidade a partir de prompts estruturados, contendo definições claras das classes, exemplos positivos e negativos e regras explícitas de decisão, garantindo maior cobertura semântica do que listas estáticas de palavras. Essa estratégia reduz o viés léxico do dicionário, captura melhor sinônimos, contexto e intenções implícitas do texto e se adapta mais rapidamente a novos sistemas, jargões ou mudanças organizacionais.
Para mitigar ruído, os rótulos produzidos pela LLM poderiam ser acompanhados de um score de confiança, permitindo filtrar apenas predições de alta confiabilidade ou combinar múltiplas chamadas (self-consistency).

Em seguida, esses rótulos serviriam como base para treinar um modelo supervisionado mais simples e eficiente (como o LinearSVC), preservando custo e latência em produção. Como evolução adicional, a LLM também poderia ser integrada ao ciclo de active learning, sendo acionada apenas para tickets com baixa confiança do modelo tradicional, funcionando como um “especialista sob demanda” e maximizando ganho de informação com controle de custo computacional.